# c04-boats

*Purpose*: Exploratory model analysis is a way to learn about a model and thus the scenario that it describes. However, computationally-expensive models are challenging to study with EMA, as every function evaluation costs us a lot of compute time. This challenge is about applying EMA ideas to learn about boat hull design, and about interfacing with a more computationally-complex simulation code.


## Informed Consent

As a reminder, this course is part of a study of engineers' behavior. While not all parts of the course are part of the study, the homeworks (challenges) are.

We will analyze your answers to this homework, and may quote this work as part of published research.

You can ask to have your responses excluded from the study after the interview by sending us an email. Please remember that this is required to participate in the professional development course. Before starting this assignment, do you consent to sharing your work with the study?


I agree to share my responses with the study

- (Please type your name here)


In [ ]:
import grama as gr
import pandas as pd
import time
DF = gr.Intention()
%matplotlib inline

# Context

This challenge comes with an additional simulation code written in Python. The code is in the file `boat_utils.py`, which you can inspect if you're interested. For the purposes of this challenge, you only need to load the simulation tools, and read the basics detailed below.


In [ ]:
# Boat analysis code
import boat_utils as boat

## Basics of Boat Hull Design

In order to float safely, a boat has to displace enough water to support its mass, and rotate back to upright as it is rolled by waves and wind. This desired behavior is governed by a surprisingly complex interplay between geometric and mass distribution factors in the boat's design. The following graphic depicts the cross-section of a simple boat hull, rotated $30^{\circ}$ from its upright position.


![Boat diagram](images/c04-boat-diagram.png)


Gravity applies a downward force at the center of mass (G), while buoyancy applies an upward force at the center of buoyancy (B'). These two forces apply a torque that rolls the boat---hopefully back towards its upright position! Naval engineers analyze this behavior using the *statical stability curve*---this is what the provided boat simulation tools compute.

The following is a statical stability curve for an example boat hull.


In [ ]:
# NOTE: No need to edit; this will analyze an example boat
t0 = time.time()
v_design = [2.5, 3.0, 2.0, 0.30, 0.35]
df_example = boat.fun_moment(v_design)
t1 = time.time()
print("Execution time: {0:4.3f} sec".format(t1 - t0))

(
    # Visualize the moment curve
    df_example
    >> gr.ggplot(gr.aes("angle * 180/3.14", "M_net"))
    + gr.geom_hline(yintercept=0, linetype="dashed")
    + gr.geom_line()
    
    + gr.scale_color_discrete(name=None)
    + gr.theme_minimal()
    + gr.labs(
        x="Boat Angle (Degrees)",
        y="Net Moment (lbf * in)",
    )
)

Some things worth noting about this example:

- The Boat Angle is the angle to which the boat hull is rotated (along its long axis).
- When the net moment is negative, a restoring torque acts to turn the boat back to upright.
- When the net moment is positive, an anti-restoring torque is applied.
<!-- - The point where the moment curve passes through the axis `M_net == 0` with positive slope is called the *angle of vanishing stability* (AVS). Rotating the boat beyond this point will cause it to settle upside-down. -->
- The boat analysis takes a non-trivial amount of time to run! Analyzing a large number of boat designs will take a fair amount of simulation time.


## Upright Stability

Note that upright stability is *not* guaranteed, even when comparing boats of identical geometry. Upright stability depends on a *restoring* moment acting when the boat is rotated away from upright.

> For a boat to be upright-stable, the slope of its moment curve must be **negative** at the angle $\theta = 0^{\circ}$.

The following boat has identical hull geometry as the previous case, but is lighter (smaller `d`) and has a higher center of mass (larger `f_com`).


In [ ]:
# NOTE: No need to edit; this will analyze an example boat
t0 = time.time()
v_light = [2.5, 3.0, 2.0, 0.20, 0.40]
df_light = boat.fun_moment(v_light)
t1 = time.time()
print("Execution time: {0:4.3f} sec".format(t1 - t0))

(
    # Visualize the moment curve
    df_light
    >> gr.tf_mutate(source="Light loading")
    >> gr.tf_bind_rows(
        df_example
        >> gr.tf_mutate(source="Baseline")
    )
    >> gr.ggplot(gr.aes("angle * 180/3.14", "M_net", color="source"))
    + gr.geom_hline(yintercept=0, linetype="dashed")
    + gr.geom_line()
    
    + gr.scale_color_discrete(name=None)
    + gr.theme_minimal()
    + gr.labs(
        x="Boat Angle (Degrees)",
        y="Net Moment (lbf * in)",
    )
)

Note that the loading conditions of the boat have destabilized it at upright; the slope of the moment curve at $0^{\circ}$ is now positive.

This phenomenon is dangerous for vessels that must operate in both heavily-loaded and lightly-loaded conditions: Imagine a large transport ship in-harbor that has just unloaded cargo. The destabilizing phenomenon we see here would put the crew and dockworkers in considerable danger.


```{admonition} Developing insight through EMA
The traditional way to learn about boat stability is to carry out a variety of analytical derivations. In this challenge, you will use computational EMA tools to build some intuition about how design variables affect boat stability.
```

## Archival Data

For this challenge, you have available a set of pre-computed simulation results. The following code loads this archival dataset.


In [ ]:
# NOTE: No need to edit; load the archival data
filename_archival = "./data/doe-wide.csv"
df_archival = pd.read_csv(filename_archival)
df_archival.head()

### Data Dictionary

There is *a lot* of information available in this dataset; the following is a summary of what's in the data:

| Column | I/O | Meaning |
|---|---|---|
| `d` | Input | Displacement ratio (-) |
| `n` | Input | Shape factor (-) |
| `f_com` | Input | Height of COM from boat bottom (-) |
| `H` | Input | Height of boat (in) |
| `W` | Input | Width of boat (in) |
|---|---|---|
| `M_max` | Output | Max righting moment (`lbf*in`) |
| `M_min` | Output | Min righting moment (`lbf*in`) |
| `mass` | Output | Boat mass (lb) |
| `dMdtheta_avs` | Output | Slope of moment curve at AVS (`lbf in/rad`) |
| `dMdtheta_0` | Output | Slope of moment curve at upright (`lbf*in/rad`) |
| `int_M_stable` | Output | Integral of moment curve, up to the AVS (`lbf*in*rad`) |
| `BM` | Output | Metacentric radius (in) |
| `GM` | Output | Metacentric height (in) |
| `angle` | Output | Angle of vanishing stability (rad) |


In [ ]:
# NOTE: No need to edit; list out the variable names
var = ["d", "n", "f_com", "H", "W"]
out = [
    "M_max", 
    "M_min", 
    "mass", 
    "dMdtheta_avs", 
    "dMdtheta_0", 
    "BM", 
    "GM",
    "angle",
    "int_M_stable",
]

# Guided EDA of Archival Data

Before going to EMA, let's start with an EDA of the archival data. This will help us frame the model work to come later.


### __q1__ First look

Use simple EDA techniques to answer the questions under *observations* below.


In [ ]:
# TASK: Perform simple EDA, answer the questions below
(
    df_archival

)

*Observations*

- How many observations (distinct boat hulls) are there in-total?
  - (Your response here)
- What are rough bounds for each of the input variables? (You'll use these in a later task....)
  - (Your response here)


### __q2__ How many designs are upright-stable?

Remember that upright-stable means the slope of the moment curve at an upright position is negative. Answer the questions under *observations* below.


In [ ]:
## TASK: Determine the number and fraction of upright-stable boats
(
    df_archival

)

*Observations*

- How many boat designs in the dataset are upright-stable?
  - (Your response here)
- What *fraction* of boat designs in the dataset are upright-stable?
  - (Your response here)


## Hypothesis: Width controls upright stability

To help frame the rest of this challenge, let's pose and assess a particular hypothesis:

> A wider boat will tend to have greater upright stability. Thus, we can make a boat stable by making it wider.

This seems intuitive, but is it true? Let's find out!


### __q3__ Correlation tile plot

Construct a correlation tile plot to study the input/output relationships. Answer the questions under *observations* below.


In [ ]:
(
    df_archival
    ## Compute the input/output correlation data here

    ## NOTE: No need to edit; this will visualize your data
    >> gr.pt_auto()
)

*Observations*

- Which inputs correlate strongly with the slope of the moment curve at upright (`dMdtheta_0`)?
  - (Your response here)
- For each of the inputs you identified above, what is the sign of the correlation? (Positive or negative?) For each input, what kind of change (increase or decrease) is required to increase upright stability?
  - (Your response here)
- How do these results relate to the hypothesis above?
  - (Your response here)


### __q4__ Detailed scatterplots

Make scatterplots to study relationship between `dMdtheta_0` and the relevant inputs you identified above. Add a smooth trend to each scatterplot with `gr.geom_smooth()`. Answer the questions under *observations* below.

*Hint*: While it is possible to construct all three plots in a single code cell, you should feel free to create additional cells as-necessary.


In [ ]:
# TODO: Create scatterplots


*Observations*

- Is the observed variability real or erroneous?
  - (Your response here)
- Describe the effects of each input on the output `dMdtheta_0`.
  - (Your response here)
- Do any of the inputs have a more *predictable* effect on `dMdtheta_0`?
  - (Your response here)
- How do these results relate to the hypothesis above?
  - (Your response here)


Having carried out an EDA of the archival data, we can move on to studying behavior using the model. However, first we need to *setup* the model.


# Model Setup

The primary tool we need from the boat simulation utilities is the performance function, called like so:

```python
v = [2.5, 3.0, 2.0, 0.30, 0.35]
df_res = boat.fun_performance(v)
```

We will use this function to compute performance metrics (such as `dMdtheta_0`) for various boat designs. However, before we do so, we should quantify how long it takes to run the performance function. Doing so will help us *plan* our EMA evaluations, so that we don't try to execute a study that will take too long.


### __q5__ Time the analysis

Evaluate the boat performance function *inside* the loop below in order to complete the timing study. You'll use this number later in `gr.cp_function()` (in q7).

*Hint*: Remember that `v_design` was assigned above in an example chunk. You can use that as the argument to `fun_performance()`.


In [ ]:
## TASK: Evaluate the boat performance to time the simulation
n_rep = 5

t0 = time.time()
for i in range(n_rep):
    # Evaluate the boat performance function here to
    # complete the timing study

t1 = time.time()

t_exec = (t1 - t0) / n_rep

print("Average execution time: {0:} sec".format(t_exec))


### __q6__ Define the input order

List the inputs to `boat.fun_performance()` in the correct order. You will use this to assemble the model below.

*Hint*: Remember to consult the documentation for `boat.fun_performance()` to see the order of its arguments.


In [ ]:
## TODO:
var_performance = []


## NOTE: No need to edit; this defines the output quantities
out_performance = [
    "mass",
    "GM",
    "angle",
    "dMdtheta_0",
    "dMdtheta_avs",
    "M_max",
    "M_min",
    "int_M_stable",
]

## NOTE: No need to edit; use this to check your work
assert \
    set(var_performance) == set(boat.var_correct), \
    "Inputs not correct"
assert \
    var_performance == boat.var_correct, \
    "Inputs are not in the correct order; check " + \
    "the documentation for boat.fun_performance()."

```{admonition} Why does order matter?
By default, Python functions interpret their inputs based on **order**. Grama takes care of variables based on **name**, but when working with a "regular" Python function, we have to be careful to give grama the correct order information. Imagine giving the value for boat height `H` as the shape factor `n`, or vice versa! If we are careful when defining a grama model, we can prevent these order issues later.
```

### __q7__ Assemble the model

Complete the code below to assemple the boat performance model. Use the values assigned above as arguments to `gr.cp_function()`, including the variable names, output names, and runtime estimate (value from **q5**). Make sure to add bounds for the inputs that "enclose" the observed archival data.

```{admonition} Why not `gr.cp_vec_function()`?
Here we use `gr.cp_function()` rather than `gr.cp_vec_function()`. This is because the boat performance function is not *vectorized*; it cannot take a DataFrame as its input. Generally we use `gr.cp_function()` with more "expensive" to evaluate functions.
```

In [ ]:
## TODO: Complete the code below
md_performance = (
    gr.Model("Boat performance")
    >> gr.cp_function(
        fun=boat.fun_performance,
        # Add input and output names;
        # runtime estimate

    )
    >> gr.cp_bounds(
        # Add input bounds

    )
)

## NOTE: No need to edit; use this to check your work
assert \
    md_performance.functions[0].runtime > 0, \
    "Model has no runtime estimate"
for v in var_performance:
    assert \
        md_performance.domain.bounds[v][0] < df_archival[v].min(), \
        "Archival data for {} exceeds model lower bound".format(v)
    assert \
        df_archival[v].max() < md_performance.domain.bounds[v][1], \
        "Archival data for {} exceeds model upper bound".format(v)

md_performance

# EMA with Expensive Models

We've seen trends in the detailed scatterplots, but we should confirm them with the real model. A sinew plot will help us confirm what we saw above and better-assess the width hypothesis.

*However*, note that the analysis code we're using has a nontrivial runtime; we should be careful about how many evaluations we request or we could be waiting a *long* time for the simulation to run. The following task will guide you through balancing runtime with exploring the model's behavior.


### __q8__ Plan a study

Tune the arguments to achieve a "reasonable" runtime. You will need to decide for yourself what a reasonable runtime is (though it will at a minimum be ~3 minutes.) Answer the questions under *observations* below.

*Hint 1*: Remember to use the documentation for a function to see which arguments to `gr.ev_sinews()` you can tune.

*Hint 2*: Running at least three sweeps will help with the task that comes after this one.


In [ ]:
## TODO: Tune the arguments for gr.ev_sinews()
# to get a "reasonable" runtime
(
    md_performance
    >> gr.ev_sinews(
        df_det="swp", # Sweep all deterministic inputs
        skip=True,    # This prevents execution!

    )
)

*Observations*

- What is the predicted runtime?
  - (Your response here)


### __q9__ Run the study

Now, use the settings you determined above and run the study! Answer the questions under *observations* below.

*Note*: It's a good idea to set this to run, go make yourself a coffee, then come back!


In [ ]:
t0 = time.time()
df_sinews = (
    md_performance
    >> gr.ev_sinews(
        df_det="swp", # Sweep all deterministic inputs
        ## Use the same settings you determined above

    )
)
t1 = time.time()
print("Execution time: {} sec".format(t1 - t0))


Once the evaluation is complete, visualize the results with the following code.


In [ ]:
# NOTE: No need to edit; use this to visualize the results
(
    df_sinews
    >> gr.pt_auto()
)

*Observations*

- How close to the predicted runtime is the actual runtime?
  - (Your response here)
- Consider the variables you identified above that affect `dMdtheta_0`. Do they have a *consistent* effect on upright stability, or does their effect depend on the value of other inputs?
  - (Your response here)
